# Notes

result_id 10040 and 10041 have None for Service Type ( change to CAF?)

result id 10002 has a null Benefit (set to Disability Pension?)

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 500

# Get Data

In [3]:
df_mapping_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=2)

# Clean Data

In [4]:
df_mapping = df_mapping_orig.rename(columns={df_mapping_orig.columns[0]: "Result ID", 
                                         "Statements": "Scenario",
                                         "Unnamed: 129": "Service Type",
                                         "Unnamed: 127": "Result Name",
                                         "Unnamed: 130": "Applicant",
                                         "Unnamed: 131": "Benefit",
                                         "Unnamed: 132": "Additional Details",
                                         "Unnamed: 128": "Result Description",
                                        })
df_mapping = df_mapping.drop(df_mapping.index[0])

In [5]:
#df_mapping[df_mapping['Service Type'].isnull()]
df_mapping['Service Type'] = df_mapping['Service Type'].fillna('CAF')

In [6]:
#df_mapping[df_mapping['Benefit'].isnull()]
df_mapping.loc[df_mapping['Result ID'] == 10002, 'Benefit'] = 'Disability Pension'

# Summarize by Result ID

Create a table with one row per result id. Have a column containing all `Applicant` and `Service Type` for that result.

In [7]:
df = df_mapping[['Result ID', 'Benefit', 'Applicant', 'Service Type']][:]
df['Applicant'] = df['Applicant'] + '; '
df['Service Type'] = df['Service Type'] + '; '
df['Benefit'] = df['Benefit'].astype(str)
df.head()

,Result ID,Benefit,Applicant,Service Type
11,1,Disability Benefits,Member;,CAF;
21,2,Disability Benefits,Member;,CAF;
31,3,Disability Benefits,Member;,War Service Veteran ;
32,3,Disability Benefits,Member;,RCMP;
33,3,Disability Benefits,Member;,Allied Veteran;


In [8]:
aggregation = {
    'Benefit': lambda x: x.unique().sum(),
    'Applicant': lambda x: x.unique().sum(),
    'Service Type':lambda x: x.unique().sum(),
}

df_mapping_summary = df.groupby(['Result ID']).agg(aggregation)
df_mapping_summary.rename(columns={'Applicant': 'Applicant Types', 'Service Type':'Service Types'}, inplace=True)
df_mapping_summary.reset_index(inplace=True)
df_mapping_summary = df_mapping_summary[['Benefit', 'Result ID', 'Applicant Types', 'Service Types']]
df_mapping_summary.set_index(['Benefit', 'Result ID'], inplace=True)
df_mapping_summary

Applicant Types  \
Benefit                                              Result ID                                                                                                               
Disability Benefits                                  1                                                                                                            Member;    
                                                     2                                                                                                            Member;    
                                                     3                                                                                                            Member;    
                                                     4                                                                                                            Member;    
                                                     5                                                                                                            Member;    
Family Caregiver Relief Benefit                      6                                                                                                            Member;    
Attendance Allowance                                 7                                                                                                            Member;    
Clothing Allowance                                   8                                                                                                            Member;    
Exceptional Incapacity Allowance                     9                                                                                                            Member;    
Veterans Hiring Act (Priority Hire)                  10                                                                                                           Member;    
Rehabilitation Services                              11                                                                                                           Member;    
                                                     12                                                                                                           Member;    
                                                     13                                                                                                           Member;    
Permanent Impairment Allowance                       14                                                                                                           Member;    
                                                     15                                                                                                           Member;    
Totally and Permanently Incapacitated                16                                                                                                           Member;    
Canadian Forces Income Supplement                    17                                                                                                           Member;    
Retirement Income Security Benefit                   18                                                                                                           Member;    
Veterans Independence Program                        19                                                                                                           Member;    
War Veterans Allowance                               20                                                                                                           Member;    
                                                     21                                                                                                           Member;    
Career Transition Services                           22                                                                                                  

## Summarize by Benefit

Create a table with one row per benefit. Have a column containing all `Applicant` and `Service Type` for that result.

In [9]:
df = df_mapping[['Benefit', 'Applicant', 'Service Type']][:]
df['Benefit'] = df['Benefit'].astype(str)
df['Applicant'] = df['Applicant'] + '; '
df['Service Type'] = df['Service Type'] + '; '

aggregation = {
    'Applicant': lambda x: x.unique().sum(),
    'Service Type':lambda x: x.unique().sum(),
}

df_mapping_summary = df.groupby(['Benefit']).agg(aggregation)
df_mapping_summary = df_mapping_summary.rename({'Applicant': 'Applicant Types', 'Service Type':'Service Types'})
df_mapping_summary

,Applicant,Service Type
Benefit,,
Attendance Allowance,Member; Spouse / Common-law Partner;,CAF; War Service Veteran ; RCMP; Allied Veteran;
Canadian Forces Income Supplement,Member; Survivor; Orphan (Veteran is deceased);,CAF;
Career Transition Services,Member;,CAF;
Case Management,Member;,CAF; RCMP; War Service Veteran ; Allied Veteran;
Clothing Allowance,Member;,CAF; War Service Veteran ; RCMP; Allied Veteran;
Critical Injury Benefit,Member;,CAF;
Death Benefit,Survivor; Orphan (Veteran is deceased);,CAF;
Disability Award,Survivor;,CAF;
Disability Benefits,Member; Spouse / Common-law Partner;,CAF; War Service Veteran ; RCMP; Allied Veteran;
